In [ ]:
!pip install tflite-model-maker

In [ ]:
!zip -r scenes-selected-5000.zip scenes-selected-5000

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Kaggle"

In [ ]:
!kaggle datasets download -d umeshsati54/scenes-selected-5000

In [ ]:
!unzip scenes-selected-5000.zip

In [ ]:
import os

import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import configs
from tflite_model_maker import ExportFormat
from tflite_model_maker import image_classifier
from tflite_model_maker import ImageClassifierDataLoader
from tflite_model_maker import model_spec

import matplotlib.pyplot as plt

In [ ]:
data = ImageClassifierDataLoader.from_folder("/content/scenes-selected-5000/scenes-selected-5000")

In [ ]:
train_data, validation_data = data.split(0.6)

In [ ]:
plt.figure(figsize=(10,10))
for i, (image, label) in enumerate(data.gen_dataset().unbatch().take(25)):
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image.numpy(), cmap=plt.cm.gray)
  plt.xlabel(data.index_to_label[label.numpy()])
plt.show()

In [ ]:
##############
model = image_classifier.create(train_data, model_spec=model_spec.efficientnet_lite4_spec, validation_data=validation_data, train_whole_model=True, shuffle=True,epochs=3)

In [ ]:
loss, accuracy = model.evaluate(test_data)

In [ ]:
def get_label_color(val1, val2):
  if val1 == val2:
    return 'black'
  else:
    return 'red'
plt.figure(figsize=(20, 20))
predicts = model.predict_top_k(test_data)
for i, (image, label) in enumerate(test_data.gen_dataset().unbatch().take(100)):
  ax = plt.subplot(10, 10, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image.numpy(), cmap=plt.cm.gray)

  predict_label = predicts[i][0][0]
  color = get_label_color(predict_label,
                          test_data.index_to_label[label.numpy()])
  ax.xaxis.label.set_color(color)
  plt.xlabel('Predicted: %s' % predict_label)
plt.show()

In [ ]:
!zip -r model_0.9481.zip model_0.9481

In [ ]:
model.export(export_dir='.', export_format=[ExportFormat.LABEL,ExportFormat.TFLITE,ExportFormat.SAVED_MODEL])

In [ ]:
model.evaluate_tflite('model.tflite', validation_data)

In [ ]:
config = configs.QuantizationConfig.create_full_integer_quantization(representative_data=validation_data, is_integer_only=True)
con = configs.QuantizationConfig.create_dynamic_range_quantization()

In [ ]:
model.export(export_dir='.', tflite_filename='model_quant_default.tflite', quantization_config=con)
model.export(export_dir='.', tflite_filename='model_quant_int8.tflite', quantization_config=config)

#TESTING TFLITE MODEL

In [ ]:
IMAGE_SHAPE = (300, 300)
TRAINING_DATA_DIR = 'scenes-selected-5000'

datagen_kwargs = dict(rescale=1./255, validation_split=.1)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
test_generator = test_datagen.flow_from_directory(TRAINING_DATA_DIR, subset="validation", shuffle=True,target_size=IMAGE_SHAPE,batch_size=100)

In [ ]:
# Learn about dataset labels
dataset_labels = sorted(test_generator.class_indices.items(), key=lambda pair:pair[1])
dataset_labels = np.array([key.title() for key, value in dataset_labels])
print(dataset_labels)

In [ ]:
# Get images and labels batch from test dataset generator
test_image_batch, test_label_batch = next(iter(test_generator))
true_label_ids = np.argmax(test_label_batch, axis=-1)
print("test batch shape:", test_image_batch.shape)

In [ ]:
#TESTING TFLITE MODEL
tflite_interpreter = tf.lite.Interpreter(model_path='model_0.9481.tflite')

input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

In [ ]:
tflite_interpreter.resize_tensor_input(input_details[0]['index'], (100, 300, 300, 3))
tflite_interpreter.resize_tensor_input(output_details[0]['index'], (100, 15))
tflite_interpreter.allocate_tensors()

input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

In [ ]:
tflite_interpreter.set_tensor(input_details[0]['index'], test_image_batch)

tflite_interpreter.invoke()

tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
print("Prediction results shape:", tflite_model_predictions.shape)


In [ ]:
# Convert prediction results to Pandas dataframe, for better visualization
import pandas as pd

tflite_pred_dataframe = pd.DataFrame(tflite_model_predictions)
tflite_pred_dataframe.columns = dataset_labels

print("TFLite prediction results for the first elements")
tflite_pred_dataframe.head()

In [ ]:
# Print images batch and labels predictions for TFLite Model
import matplotlib.pyplot as plt

tflite_predicted_ids = np.argmax(tflite_model_predictions, axis=-1)
tflite_predicted_labels = dataset_labels[tflite_predicted_ids]
tflite_label_id = np.argmax(test_label_batch, axis=-1)

plt.figure(figsize=(15,15))
plt.subplots_adjust(hspace=0.5)
for n in range(100):
  plt.subplot(10,10,n+1)
  plt.imshow(test_image_batch[n])
  color = "green" if tflite_predicted_ids[n] == true_label_ids[n] else "red"
  plt.title(tflite_predicted_labels[n].title(), color=color)
  plt.axis('off')
_ = plt.suptitle("TFLite model predictions (green: correct, red: incorrect)")

##Performance Matrices for 10000 images

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle

In [ ]:
!kaggle datasets download -d binsbins/scenes-test-dataset

In [ ]:
!unzip archive.zip

In [ ]:
import os
import numpy as np
import tensorflow as tf

IMAGE_SHAPE = (300, 300)
TRAINING_DATA_DIR = '/content/test'

datagen_kwargs = dict(rescale=1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
test_generator = test_datagen.flow_from_directory(TRAINING_DATA_DIR, shuffle=True,target_size=IMAGE_SHAPE,batch_size=1)


# Learn about dataset labels
dataset_labels = sorted(test_generator.class_indices.items(), key=lambda pair:pair[1])
dataset_labels = np.array([key.title() for key, value in dataset_labels])
print(dataset_labels)


true_= None
for i in range(7577):
  # Get images and labels batch from test dataset generator
  test_image_batch, test_label_batch = next(iter(test_generator))
  if i==0:
    true_ = test_label_batch
  else:
    true_ = np.append(true_, test_label_batch, axis=0)
  true_label_ids = np.argmax(test_label_batch, axis=-1)
  #print("test batch shape:", test_image_batch.shape)
  print('Testing on ',(i+1)*1)

  tflite_interpreter = tf.lite.Interpreter(model_path='model.tflite')

  input_details = tflite_interpreter.get_input_details()
  output_details = tflite_interpreter.get_output_details()

  tflite_interpreter.resize_tensor_input(input_details[0]['index'], (1, 300, 300, 3))
  tflite_interpreter.resize_tensor_input(output_details[0]['index'], (1, 15))
  tflite_interpreter.allocate_tensors()

  input_details = tflite_interpreter.get_input_details()
  output_details = tflite_interpreter.get_output_details()

  tflite_interpreter.set_tensor(input_details[0]['index'], test_image_batch)

  tflite_interpreter.invoke()
  if i==0:
    tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
  else:
    temp = tflite_interpreter.get_tensor(output_details[0]['index'])
    tflite_model_predictions = np.append(tflite_model_predictions, temp, axis=0)
  
  del tflite_interpreter

In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

tflite_predicted_ids1 = np.argmax(tflite_model_predictions, axis=-1)
true_label_ids1 = np.argmax(true_, axis=-1)

y_true = dataset_labels[np.argmax(true_, axis=-1)]
y_pred = dataset_labels[np.argmax(tflite_model_predictions, axis=-1)]
data = confusion_matrix(y_true, y_pred)
df_cm = pd.DataFrame(data, columns=np.unique(y_true), index = np.unique(y_true))
df_cm.index.name = 'Predicted'
df_cm.columns.name = 'Actual'
plt.figure(figsize = (17,15))
sn.set(font_scale=1.4)#for label size
sn.heatmap(df_cm, cmap="Blues", annot=True,annot_kws={"size": 12},fmt='g')# font size

## ACCURACY FOR EACH CLASS

In [ ]:
accu = {}
tflite_predicted_ids1 = np.argmax(tflite_model_predictions, axis=-1)
true_label_ids1 = np.argmax(true_, axis=-1)
for i in range(len(true_label_ids1)):
  if true_label_ids1[i] == tflite_predicted_ids1[i]:
    accu[str(dataset_labels[true_label_ids1[i]])] = 0

for i in range(len(true_label_ids1)):
  if true_label_ids1[i] == tflite_predicted_ids1[i]:
    accu[str(dataset_labels[true_label_ids1[i]])] += 1

pred = list(tflite_predicted_ids1)
true = list(true_label_ids1)
for i in range(15):
  print(dataset_labels[i],'-', round(accu[dataset_labels[i]]/true.count(i)*100,2),'%')

#print(accu)

##ROC CURVE FOR EACH CLASS

In [ ]:
from sklearn.metrics import roc_curve, auc

def plot_roc_curve(fpr,tpr,title): 
  plt.plot(fpr,tpr) 
  plt.axis([0,1,0,1]) 
  plt.title(title)
  plt.xlabel('False Positive Rate') 
  plt.ylabel('True Positive Rate') 
  plt.show()    
  
for i in range(15):
  title = dataset_labels[i]
  fpr, tpr, thresholds = roc_curve(np.argmax(true_, axis=-1), tflite_model_predictions.max(axis=1)[:,None], pos_label=i)
  plot_roc_curve (fpr,tpr,title)

##F1 score

In [ ]:
#https://www.tensorflow.org/api_docs/python/tf/keras/metrics/AUC
import tensorflow_addons as tfa
metric = tfa.metrics.F1Score(num_classes=15, threshold=0.5)

metric.update_state(true_, tflite_model_predictions)

result = metric.result()
result.numpy()

#['Bathroom' 'Bedroom' 'Bridge' 'Cafeteria' 'Classroom'
# 'Computer_Room' 'Dining_Hall' 'Hospital_Room' 'Kitchen' 'Library_Indoor'
# 'Parking_Lot' 'Playground' 'River' 'Shopfront' 'Supermarket']

##Precision

In [ ]:
for i in range(15):
  m = tf.keras.metrics.Precision(class_id=i)
  m.update_state(true_, tflite_model_predictions)
  print(m.result().numpy())

##Recall

In [ ]:
for i in range(15):
  m = tf.keras.metrics.Recall(class_id=i)
  m.update_state(true_, tflite_model_predictions)
  print(m.result().numpy())

##Mean Absolute Error

In [ ]:
m = tf.keras.metrics.MeanAbsoluteError()
m.update_state(true_, tflite_model_predictions)
m.result().numpy()

##Mean Squared Error

In [ ]:
m = tf.keras.metrics.MeanSquaredError()
m.update_state(true_, tflite_model_predictions)
m.result().numpy()

## Area Under Curve

In [ ]:
m = tf.keras.metrics.AUC(num_thresholds=3)
m.update_state(true_, tflite_model_predictions)
m.result().numpy()

##Confusion Matrix


*   true negatives for class i in M(0,0)
*   false positives for class i in M(0,1)

*   false negatives for class i in M(1,0)
*   true positives for class i in M(1,1)


In [ ]:
metric = tfa.metrics.MultiLabelConfusionMatrix(num_classes=15)
metric.update_state(true_, (tflite_model_predictions == tflite_model_predictions.max(axis=1)[:,None]).astype(int))
result = metric.result()
result.numpy().astype(int)

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

arr = result.numpy().astype(int)

labels = ['Bathroom' ,'Bedroom' ,'Bridge' ,'Cafeteria' ,'Classroom' ,'Computer_Room'
 ,'Dining_Hall' ,'Hospital_Room' ,'Kitchen' ,'Library_Indoor' ,'Parking_Lot'
 ,'Playground' ,'River' ,'Shopfront' ,'Supermarket']

fig,ax = plt.subplots(ncols=3,nrows=5)
fig.set_size_inches(15,15)

for i in range(15):
  ax_ = ax.flat[i]
  df_cm = pd.DataFrame(arr[i], range(2), range(2))
  sn.heatmap(df_cm, annot=True, annot_kws={"size": 16},fmt='g',ax=ax_) # font size
  ax_.set_title(labels[i])
  #plt.show()

##Test on Single Image

In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import pathlib

dataset_labels = ['Bathroom', 'Bedroom', 'Bridge', 'Cafeteria', 'Classroom', 'Computer_Room', 'Dining_Hall', 'Hospital_Room', 'Kitchen', 'Library_Indoor', 'Parking_Lot','Playground', 'River', 'Shopfront', 'Supermarket']

interpreter = tf.lite.Interpreter(model_path="model_0.9481.tflite")

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

interpreter.allocate_tensors()

img_path = '/content/parking-in-philadelphia.jpg'
img = cv2.imread(img_path)
img = np.array(img, dtype=np.float32)
new_img = cv2.resize(img, (300, 300))

interpreter.set_tensor(input_details[0]['index'], [new_img])
interpreter.invoke()

output_data = interpreter.get_tensor(output_details[0]['index'])
tflite_predicted_ids = np.argmax(output_data, axis=-1)
flite_predicted_labels = dataset_labels[tflite_predicted_ids[0]]
print("For file {}, the output is {}".format(img_path, flite_predicted_labels))